# Viewing Data

In [61]:
%maven org.jfree:jfreechart:1.5.3
%maven com.opencsv:opencsv:5.5
import org.jfree.chart.ChartFactory;
import org.jfree.chart.ChartUtils;
import org.jfree.chart.JFreeChart;
import org.jfree.data.category.CategoryDataset;
import org.jfree.data.category.DefaultCategoryDataset;
import java.io.File;
import java.io.BufferedReader;  
import java.io.IOException;
import java.io.FileReader;

public class JFreeChartExample {

    public static CategoryDataset createDataset(String ticker){
        DefaultCategoryDataset dataset = new DefaultCategoryDataset();
        String csvFilePath = "./stock_data/" + ticker + ".csv";
        try (BufferedReader br = new BufferedReader(new FileReader(csvFilePath))) {
            int dateColumnIndex = 0;
            int closeColumnIndex = 6;
            String line = br.readLine();  
            while ((line = br.readLine()) != null) {
                String[] data = line.split(",");
                String date = data[dateColumnIndex];
                Double close = Double.parseDouble(data[closeColumnIndex]);
                dataset.addValue(close, ticker, date);
            }
        } catch (IOException e) {
            e.printStackTrace();
        }

        return dataset;
    }

    public static JFreeChart createChart(CategoryDataset dataset, String ticker, String val1, String val2) {
        String title = val1 + " vs. " + val2 + " for " + ticker;
        return ChartFactory.createLineChart(
                title,
                val1,
                val2,
                dataset
        );
    }

    public static void saveChartAsImage(JFreeChart chart, String filePath, int width, int height) throws IOException {
        ChartUtils.saveChartAsPNG(new File(filePath), chart, width, height);
    }

    public static void main(String[] args) {
        try {
            String[] tickers = {"AAPL", "AMZN", "COST", "GOOGL", "LMT", "META", "MSFT", "NOC", "TSLA", "UNH", "WMT"};
            for (String ticker : tickers) {
                CategoryDataset dataset = createDataset(ticker);
                JFreeChart chart = createChart(dataset, ticker, "Date", "Volume");
                String chartname = "./stock_charts/date_vs_volume/" + ticker + "_chart.png";

                // Save chart as image
                saveChartAsImage(chart, chartname, 800, 600);

                System.out.println("Chart saved as image: " + chartname);
            }
            
        } catch (IOException e) {
            e.printStackTrace();
        }
    }
}

JFreeChartExample.main(null);


Chart saved as image: ./stock_charts/date_vs_volume/AAPL_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/AMZN_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/COST_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/GOOGL_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/LMT_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/META_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/MSFT_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/NOC_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/TSLA_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/UNH_chart.png
Chart saved as image: ./stock_charts/date_vs_volume/WMT_chart.png


# Create testing and training datasets

In [1]:
%maven org.nd4j:nd4j-native-platform:1.0.0-beta2
%maven org.deeplearning4j:deeplearning4j-core:1.0.0-beta2
%maven org.deeplearning4j:deeplearning4j-ui_2.11:1.0.0-beta2
%maven ch.qos.logback:logback-classic:1.2.3

In [21]:
import java.util.ArrayList;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
import org.nd4j.linalg.dataset.api.iterator.StandardScaler;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.dataset.api.preprocessor.NormalizerMinMaxScaler;

public class DatasetCreator {
    String ticker;
    String tickerPath;
    
    public DatasetCreator(String ticker) {
        this.ticker = ticker;
        this.tickerPath = "./stock_data/" + this.ticker + ".csv";
    }

    private double[] extractCSVCloseData(){
        ArrayList<Double> CSVData = new ArrayList<Double>();
        try (BufferedReader br = new BufferedReader(new FileReader(this.tickerPath))) {
            int closeColumnIndex = 4;
            String line = br.readLine();  
            while ((line = br.readLine()) != null) {
                String[] data = line.split(",");
                CSVData.add(Double.parseDouble(data[closeColumnIndex]));
            }
        } catch (IOException e) {
            e.printStackTrace();
        }

        double[] CSVDataArray = new double[CSVData.size()];
        for (int i = 0; i < CSVData.size(); i++) {
            CSVDataArray[i] = CSVData.get(i);
        }

        return CSVDataArray;
    }

    private INDArray createScaledData() {
        double[] CSVDataArray = this.extractCSVCloseData();
        INDArray data = Nd4j.create(CSVDataArray);
        // NormalizerMinMaxScaler scaler = new NormalizerMinMaxScaler(0, 1);
        // scaler.fit(data);
        // INDArray scaledData = scaler.transform(data);
        return data;
    }

    public void main(String args[]) {
        INDArray scaledData = createScaledData();
        System.out.println(scaledData);
    }
}

DatasetCreator test = new DatasetCreator("AAPL");
test.main(null);

EvalException: Could not initialize class org.nd4j.linalg.factory.Nd4j

# Testing code for LSTM

In [4]:
import org.deeplearning4j.datasets.iterator.impl.*;
import org.deeplearning4j.nn.api.OptimizationAlgorithm;
import org.deeplearning4j.nn.conf.GradientNormalization;
import org.deeplearning4j.nn.conf.NeuralNetConfiguration;
import org.deeplearning4j.nn.conf.layers.LSTM;
import org.deeplearning4j.nn.conf.layers.RnnOutputLayer;
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
import org.deeplearning4j.nn.weights.WeightInit;
import org.deeplearning4j.optimize.listeners.ScoreIterationListener;
import org.nd4j.linalg.activations.Activation;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;
import org.nd4j.linalg.learning.config.Adam;
import org.nd4j.linalg.lossfunctions.LossFunctions;
import java.util.ArrayList;
import java.util.List;

In [6]:
public class LstmExample {

    public static void main(String[] args) {
        // Define your dataset and load it
        // Example: List<double[]> features = loadFeatures();
        // List<double[]> labels = loadLabels();

        // Create DataSetIterator
        DataSetIterator iterator = new ListDataSetIterator<>(createData(features, labels));

        // Neural network configuration
        MultiLayerNetwork model = new MultiLayerNetwork(new NeuralNetConfiguration.Builder()
                .seed(123)
                .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
                .updater(new Adam(0.01))
                .l2(1e-4)
                .weightInit(WeightInit.XAVIER)
                .gradientNormalization(GradientNormalization.ClipElementWiseAbsoluteValue)
                .gradientNormalizationThreshold(0.5)
                .list()
                .layer(new LSTM.Builder()
                        .nIn(numInputs)
                        .nOut(numHiddenUnits)
                        .activation(Activation.TANH)
                        .build())
                .layer(new RnnOutputLayer.Builder(LossFunctions.LossFunction.MSE)
                        .activation(Activation.IDENTITY)
                        .nIn(numHiddenUnits)
                        .nOut(numOutputs)
                        .build())
                .pretrain(false)
                .backprop(true)
                .build()
        );

        model.init();
        model.setListeners(new ScoreIterationListener(20));  // Print the score with every 20 iterations

        // Train the model
        int numEpochs = 50;
        for (int i = 0; i < numEpochs; i++) {
            model.fit(iterator);
        }

        // Your model is now trained and ready for prediction
    }

    private static List<DataSet> createData(List<double[]> features, List<double[]> labels) {
        List<DataSet> dataSets = new ArrayList<>();
        for (int i = 0; i < features.size(); i++) {
            dataSets.add(new DataSet(Nd4j.create(features.get(i)), Nd4j.create(labels.get(i))));
        }
        return dataSets;
    }

    // Implement methods to load your features and labels from your dataset
    // private static List<double[]> loadFeatures() { ... }
    // private static List<double[]> loadLabels() { ... }
}

CompilationException: 